In [63]:
import requests
#import mysql.connector
import os
#from dotenv import load_dotenv
#import json
#import pandas as pd
import pymysql
import time
import cryptography

# Configuración API
URL = "https://cramer.buk.cl/api/v1/chile/employees"
TOKEN = "nbeU1PWnHXqXJLFGzoh5yiyw"


# Configuración BD
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "lucerna4925"
DB_NAME = "datos_buk_rrhh"

# Imprimir las variables de entorno
# print(f"TOKEN: {os.getenv('BUK_AUTH_TOKEN')}")
# print(f"SQL: {os.getenv('clave_sql')}")


In [86]:
def obtener_todos_los_empleados():
    """
    Función que obtiene los empleados limitando la búsqueda a 4 páginas.
    """
    headers = {"auth_token": TOKEN}
    todos_los_empleados = []
    url_actual = URL
    pagina_actual = 1
    
    print("🚀 Comenzando la obtención de empleados con paginación...")
    
    # Condición para limitar a 4 páginas
    while url_actual and pagina_actual <= 4:
        print(f"📄 Obteniendo página {pagina_actual}...")
        
        try:
            respuesta = requests.get(url_actual, headers=headers)
            respuesta.raise_for_status()  # Lanza una excepción para errores HTTP
            
            respuesta_api = respuesta.json()
            empleados_pagina = respuesta_api['data']
            pagination_info = respuesta_api['pagination']
            
            # Agregar empleados de esta página a la lista total
            todos_los_empleados.extend(empleados_pagina)
            
            print(f"✅ Página {pagina_actual}: {len(empleados_pagina)} empleados obtenidos")
            print(f"📊 Total acumulado: {len(todos_los_empleados)} empleados")
            
            # Obtener la URL de la siguiente página
            url_actual = pagination_info.get('next')
            pagina_actual += 1
            
            # Pequeña pausa para no sobrecargar la API
            time.sleep(0.5)
            
        except requests.exceptions.RequestException as e:
            print(f"❌ Error en la petición: {e}")
            break # Salir del bucle si hay un error
            
    print(f"🎉 ¡Paginación completada! Total de empleados obtenidos: {len(todos_los_empleados)}")
    return todos_los_empleados

# Obtener todos los empleados
empleados = obtener_todos_los_empleados()

🚀 Comenzando la obtención de empleados con paginación...
📄 Obteniendo página 1...
✅ Página 1: 25 empleados obtenidos
📊 Total acumulado: 25 empleados
📄 Obteniendo página 2...
✅ Página 2: 25 empleados obtenidos
📊 Total acumulado: 50 empleados
📄 Obteniendo página 3...
✅ Página 3: 25 empleados obtenidos
📊 Total acumulado: 75 empleados
📄 Obteniendo página 4...
✅ Página 4: 25 empleados obtenidos
📊 Total acumulado: 100 empleados
🎉 ¡Paginación completada! Total de empleados obtenidos: 100


In [ ]:
#
def obtener_todos_los_empleados():
    """
    Función que obtiene todos los empleados iterando por todas las páginas
    """
    headers = {"auth_token": TOKEN}
    todos_los_empleados = []
    url_actual = URL
    pagina_actual = 1
    
    print("🚀 Comenzando la obtención de empleados con paginación...")
    
    while url_actual and pagina_actual <= 5:
        print(f"📄 Obteniendo página {pagina_actual}...")
        
        try:
            respuesta = requests.get(url_actual, headers=headers)
            respuesta.raise_for_status()  # Lanza una excepción para errores HTTP
            #respuesta.status_code == 200:

            respuesta_api = respuesta.json()
            empleados_pagina = respuesta_api['data']
            pagination_info = respuesta_api['pagination']
            
            # Agregar empleados de esta página a la lista total
            todos_los_empleados.extend(empleados_pagina)
            
            print(f"✅ Página {pagina_actual}: {len(empleados_pagina)} empleados obtenidos")
            print(f"📊 Total acumulado: {len(todos_los_empleados)} empleados")
            print(f"📈 Páginas restantes: {pagination_info['total_pages'] - pagina_actual}")
            
            # Verificar si hay una página siguiente
            url_actual = pagination_info.get("previous: 1")
            pagina_actual += 1
      
            # Pequeña pausa para no sobrecargar la API
            time.sleep(0.5)
        
        else:
             print(f"❌ Error en la petición página {pagina_actual}:", respuesta.status_code)    
        break
    print(f"🎉 ¡Paginación completada! Total de empleados obtenidos: {len(todos_los_empleados)}")
    return todos_los_empleados

# Obtener todos los empleados
empleados = obtener_todos_los_empleados()

SyntaxError: expected 'except' or 'finally' block (4140992005.py, line 38)

In [65]:
print (empleados)

[{'person_id': 4232, 'id': 4232, 'picture_url': 'https://bukwebapp-enterprise-chile.s3.amazonaws.com/cramer/person/picture_url/4232/544f7e72-106d-4246-a779-adfd6adb7790-123577914.jpg.jpg', 'first_name': 'Cristian Transito', 'surname': 'Abarca', 'second_surname': 'Garrido', 'full_name': 'Cristian Transito Abarca Garrido', 'document_type': 'rut', 'document_number': '12.357.791-4', 'rut': '12.357.791-4', 'code_sheet': 'F1', 'email': 'abarcacristian0@gmail.com', 'personal_email': 'abarcacristian0@gmail.com', 'address': 'Pasaje Boston 1180', 'street': '', 'street_number': '', 'office_number': '', 'city': '', 'district': 'Maipú', 'location_id': 334, 'region': 'RM: Metropolitana de Santiago', 'office_phone': '', 'phone': '991294377', 'gender': 'M', 'birthday': '1973-02-27', 'university': None, 'degree': None, 'active_since': '2020-02-04', 'created_at': '2021-10-19T20:59:28.118-03:00', 'status': 'activo', 'payment_method': 'Transferencia Bancaria', 'payment_period': None, 'advance_payment': No

In [88]:
print("🚀 Conectando a MySQL...")
conexion = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    #charset='utf8mb4'
)
cursor = conexion.cursor()

# Crear base de datos si no existe
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
cursor.execute(f"USE {DB_NAME}")


print(f"✅ Conectado a MySQL y usando la base: {DB_NAME}")

print("🚀 Creando tabla si no existe...")
sql_create_table = """
CREATE TABLE IF NOT EXISTS empleados (
    id INT PRIMARY KEY,
    first_name VARCHAR(255),
    surname VARCHAR(255),
    full_name VARCHAR(255),
    rut VARCHAR(50),
    email VARCHAR(255),
    phone VARCHAR(50)
);
"""
cursor.execute(sql_create_table)


🚀 Conectando a MySQL...
✅ Conectado a MySQL y usando la base: datos_buk_rrhh
🚀 Creando tabla si no existe...


0

In [89]:
print("🚀 Insertando TODOS los empleados en la tabla SQL...")
contador = 0
errores = 0

for e in empleados:
    try:
        sql = """
        INSERT INTO empleados (id, first_name, surname, full_name, rut, email, phone)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            first_name=VALUES(first_name),
            surname=VALUES(surname),
            full_name=VALUES(full_name),
            rut=VALUES(rut),
            email=VALUES(email),
            phone=VALUES(phone)
        """
        values = (
            e.get("id"),
            e.get("first_name"),
            e.get("surname"),
            e.get("full_name"),
            e.get("rut"),
            e.get("email"),
            e.get("phone")
        )
        cursor.execute(sql, values)
        contador += 1
        
        # Mostrar progreso cada 100 registros
        if contador % 100 == 0:
            print(f"📝 Procesados {contador} empleados...")
            
    except Exception as error:
        print(f"⚠️ Error insertando empleado {e.get('id', 'N/A')}: {error}")
        errores += 1

conexion.commit()
print(f"✅ {contador} registros insertados/actualizados en MySQL.")
if errores > 0:
    print(f"⚠️ Se produjeron {errores} errores durante la inserción.")

🚀 Insertando TODOS los empleados en la tabla SQL...
📝 Procesados 100 empleados...
✅ 100 registros insertados/actualizados en MySQL.


In [90]:
print("🚀 Consultando estadísticas de la tabla...")
cursor.execute("SELECT COUNT(*) as total FROM empleados;")
total_registros = cursor.fetchone()[0]
print(f"📊 Total de empleados en la base de datos: {total_registros}")

print("\n🔍 Mostrando los primeros 5 empleados:")
cursor.execute("SELECT id, full_name, email FROM empleados LIMIT 5;")
for row in cursor.fetchall():
    print(f"ID: {row[0]}, Nombre: {row[1]}, Email: {row[2]}")

print("\n🔍 Mostrando los últimos 5 empleados:")
cursor.execute("SELECT id, full_name, email FROM empleados ORDER BY id DESC LIMIT 5;")
for row in cursor.fetchall():
    print(f"ID: {row[0]}, Nombre: {row[1]}, Email: {row[2]}")

🚀 Consultando estadísticas de la tabla...
📊 Total de empleados en la base de datos: 100

🔍 Mostrando los primeros 5 empleados:
ID: 4139, Nombre: Flaminio Del Carmen Agurto Urrutia, Email: agurtoflaminio@gmail.com
ID: 4141, Nombre: Jose Aguilar Gonzalez, Email: jfaguilargonza@gmail.com
ID: 4148, Nombre: Pablo Barrales Palma, Email: pablo_barrales@hotmail.com
ID: 4149, Nombre: Claudio Azua Mardones, Email: cazua@cramer.cl
ID: 4201, Nombre: Fabiola Andrea Alfaro Guzman, Email: fabby2410@gmail.com

🔍 Mostrando los últimos 5 empleados:
ID: 11419, Nombre: Nero Angelo Esteban Antihuala Zuñiga, Email: nerozuniga@gmail.com
ID: 11121, Nombre: Giovanni Araya Curín, Email: gi.arayac@gmail.com
ID: 11055, Nombre: Claudio Avaria Reyes, Email: avariaton@gmail.com
ID: 10758, Nombre: Alejandro Andrés Ayala Salas, Email: aa43680045@gmail.com
ID: 10626, Nombre: Jorge Iván Amigo Oliva, Email: jorgeamigooliva@gmail.com


In [ ]:
cursor.close()
conexion.close()
print("✅ Conexión cerrada correctamente.")

✅ Conexión cerrada correctamente.
